# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
!pip install chromadb

In [3]:
%pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
Note: you may need to restart the kernel to use updated packages.


In [4]:

from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [26]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [27]:
llm = OpenAI(temperature=0)

In [28]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "sonnets.txt")

In [30]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

In [31]:
sonnets = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [32]:
from langchain_community.document_loaders import WebBaseLoader

In [33]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [34]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 1051, which is longer than the specified 1000


## Create the Agent

In [35]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [36]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about Shakespeare's sonnets. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [37]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [39]:
agent.invoke(
    "What are the main themes in Shakespeare's sonnets?"
)



> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: "What are the main themes in Shakespeare's sonnets?"
Observation:  The main themes in Shakespeare's sonnets include love, beauty, mortality, time, and the power of poetry. Other recurring themes include jealousy, betrayal, and the complexities of human relationships.
Thought: I now know the final answer.
Final Answer: The main themes in Shakespeare's sonnets are love, beauty, mortality, time, and the power of poetry. Other recurring themes include jealousy, betrayal, and the complexities of human relationships.

> Finished chain.


{'input': "What are the main themes in Shakespeare's sonnets?",
 'output': "The main themes in Shakespeare's sonnets are love, beauty, mortality, time, and the power of poetry. Other recurring themes include jealousy, betrayal, and the complexities of human relationships."}

In [40]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 It's important to understand the differences between tools before making a decision.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff and Flake8 are both linters for Python code, but they have some key differences. Ruff uses different rule codes and prefixes than Flake8, which helps minimize conflicts across plugins and allows for easier toggling of individual plugins. Ruff also has a larger rule set, with over 800 rules compared to Flake8's ~409 rules. However, Ruff does not support custom lint rules, while Flake8 does. Additionally, Ruff is not a "pure" drop-in replacement for Flake8, as they enforce different sets of rules. Ruff also has a formatter that is designed to be a drop-in replacement for Black, while Flake8 does not have a built-in formatter.
Thought: I now know the differences between ruff and flake8.
Final Answer: Ruff has a larger rule set and a built-in formatter, but do

{'input': 'Why use ruff over flake8?',
 'output': 'Ruff has a larger rule set and a built-in formatter, but does not support custom lint rules like Flake8 does.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [46]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about Shakespeare's sonnets. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [47]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [48]:
agent.invoke(
    "What are the main themes in Shakespeare's sonnets?"
)



> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: "What are the main themes in Shakespeare's sonnets?"
Observation:  The main themes in Shakespeare's sonnets include love, beauty, mortality, time, and the power of poetry. Other recurring themes include jealousy, betrayal, and the complexities of human relationships.


> Finished chain.


{'input': "What are the main themes in Shakespeare's sonnets?",
 'output': " The main themes in Shakespeare's sonnets include love, beauty, mortality, time, and the power of poetry. Other recurring themes include jealousy, betrayal, and the complexities of human relationships."}

In [45]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 It's important to understand the differences between tools before making a decision.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff and Flake8 are both linters for Python code, but they have some key differences. Ruff uses different rule codes and prefixes than Flake8, which helps minimize conflicts across plugins and allows for easier toggling of individual plugins. Ruff also has a larger rule set, with over 800 rules compared to Flake8's ~409 rules. However, Ruff does not support custom lint rules, while Flake8 does. Additionally, Ruff is not a "pure" drop-in replacement for Flake8, as they enforce different sets of rules. Ruff also has a formatter that is designed to be a drop-in replacement for Black, while Flake8 does not have a built-in formatter.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': ' Ruff and Flake8 are both linters for Python code, but they have some key differences. Ruff uses different rule codes and prefixes than Flake8, which helps minimize conflicts across plugins and allows for easier toggling of individual plugins. Ruff also has a larger rule set, with over 800 rules compared to Flake8\'s ~409 rules. However, Ruff does not support custom lint rules, while Flake8 does. Additionally, Ruff is not a "pure" drop-in replacement for Flake8, as they enforce different sets of rules. Ruff also has a formatter that is designed to be a drop-in replacement for Black, while Flake8 does not have a built-in formatter.'}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [55]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about Shakespeare's sonnets. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
        return_direct=True,
    ),
]

In [50]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [56]:
agent2 = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=3,
    handle_parsing_errors=True
)


In [51]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the Shakespeare's mention that tool in the sonnets?"
)



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer the first question and the Sonnets QA System to answer the second question.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff does not currently have support for Jupyter Notebooks.


> Finished chain.


{'input': "What tool does ruff use to run over Jupyter Notebooks? Did the Shakespeare's mention that tool in the sonnets?",
 'output': ' Ruff does not currently have support for Jupyter Notebooks.'}

In [57]:
agent2.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the Shakespeare's mention that tool in the sonnets?"
)




> Entering new AgentExecutor chain...
Action:
{
  "action": "Ruff QA System",
  "action_input": "What tool does ruff use to run over Jupyter Notebooks?"
}



> Finished chain.


{'input': "What tool does ruff use to run over Jupyter Notebooks? Did the Shakespeare's mention that tool in the sonnets?",
 'output': 'Action:\n{\n  "action": "Ruff QA System",\n  "action_input": "What tool does ruff use to run over Jupyter Notebooks?"\n}\n\n'}